In [1]:
import sys
import json
import time
import pandas as pd
from sklearn.cross_validation import train_test_split

In [6]:
def read_dict(file_name):
    with open(file_name, encoding='utf8', errors='ignore') as file:
        obj = json.loads(file.read())
    return obj


def save_dict(obj, file_dir):
    f = open(file_dir, mode='w', encoding='utf8', errors='ignore')
    s = json.dumps(obj, indent=4, ensure_ascii=False)
    f.write(s)
    f.close()


def print_schedule(begin, i, s_=None):
    if not s_:
        return 0
    if i % 1000 == 0:
        sum_time = '%0.2f' % (time.time() - begin)
        sys.stderr.write(("\r%s %d sum time %s" % (s_, i, sum_time)))
        sys.stderr.flush()


def complete_schedual():
    sys.stderr.write("\n")
    sys.stderr.flush()

In [2]:
rating_file = 'input/ratings.dat'
gender_file = 'input/gender.dat'

In [4]:
begin = time.time()

# 性别字典
with open(gender_file) as file:
    gender_dict = dict()
    for row in file:
        user_id, gender = row.strip().split(',')
        gender_dict[user_id] = gender

print('gender dict', time.time() - begin)
sys.stderr.flush()

gender dict 0.1980276107788086


In [7]:
with open(rating_file) as file:
    male_rating, female_rating = 0, 0
    male_rating_dict = dict()
    famale_rating_dict = dict()
    i = 0
    for row in file:
        user_id, item_id, rate = row.strip().split(',')
        rate = int(rate)
        if rate >= 0:
            user = gender_dict[user_id] + str(user_id)
            item = gender_dict[item_id] + str(item_id)
            if user[0] == 'M' and item[0] == 'F':
                male_rating += 1
                # if user[0] == 'M':
                if user not in male_rating_dict:
                    male_rating_dict[user] = dict()
                male_rating_dict[user][item] = rate
            elif user[0] == 'F' and item[0] == 'M':
                female_rating += 1
                # elif user[0] == 'F':
                if item not in famale_rating_dict:
                    famale_rating_dict[item] = dict()
                famale_rating_dict[item][user] = rate
            # else:
            #     print(user, item)
        print_schedule(begin, i, 'reading rating file')
        i += 1
    complete_schedual()


print('rating dict', time.time() - begin)

reading rating file 17359000 sum time 144.60


rating dict 144.6043496131897


In [13]:
# 从评分中构造匹配字典 匹配列表
match_dict = dict()
match_list = list()
female = 0
male = 0
i = 0
neg_pair_list = list()
for user, item_rate_dict in male_rating_dict.items():
    for item, rate in item_rate_dict.items():
        if user in famale_rating_dict:
            if item in famale_rating_dict[user]:
                rate_ = famale_rating_dict[user][item]
                if rate == 10 and rate_ == 10:
                    if user not in match_dict:
                        match_dict[user] = dict()
                    match_dict[user][item] = rate * rate_
                    match_list.append([user, item, rate * rate_])
                else:
                    neg_pair_list.append([user, item, rate * rate_])
    print_schedule(begin, i, 'reading male dict')
    i += 1
complete_schedual()

reading male dict 60000 sum time 4414.77


In [14]:
print(len(match_list))

53376


In [16]:
# 从评分中构造匹配字典 匹配列表
match_dict = dict()
match_list = list()
female = 0
male = 0
i = 0
neg_pair_list = list()
for user, item_rate_dict in male_rating_dict.items():
    for item, rate in item_rate_dict.items():
        if user in famale_rating_dict:
            if item in famale_rating_dict[user]:
                rate_ = famale_rating_dict[user][item]
                if rate >= 0 and rate_ >= 0:
                    if user not in match_dict:
                        match_dict[user] = dict()
                    match_dict[user][item] = rate * rate_
                    match_list.append([user, item, rate * rate_])
                else:
                    neg_pair_list.append([user, item, rate * rate_])
    print_schedule(begin, i, 'reading male dict')
    i += 1
complete_schedual()
print(len(match_list))

reading male dict 60000 sum time 4511.45


94883


In [17]:
male_rating_frame = pd.DataFrame(male_rating_dict).fillna(1)
female_rating_frame = pd.DataFrame(female_rating_dict).fillna(1)

inter_female = list(set(male_rating_frame.index) & set(female_rating_frame.index))
inter_male = list(set(male_rating_frame.columns) & set(female_rating_frame.columns))

male_rating_frame = male_rating_frame.reindex(index=inter_female, columns=inter_male)
female_rating_frame = female_rating_frame.reindex(index=inter_female, columns=inter_male)

male_rating_frame_adj = male_rating_frame.fillna(0) * female_rating_frame.fillna(1)
female_rating_frame_adj = female_rating_frame.fillna(0) * male_rating_frame.fillna(1)


MemoryError: 